In [ ]:
# Might actually end up using a Jupyter notebook

In [21]:
# import modules
from selenium.webdriver import Firefox
import time
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
import re
from tkinter import Tk
from tkinter.filedialog import askopenfilename
import datetime
from shutil import copyfile
import yfinance

# import the csv file:

# root = Tk()
# root.withdraw()
# source_data = askopenfilename()
# for now, just use the direct file path
stock_df = pd.read_csv('test_data.csv', index_col=False)
stock_df = stock_df.sort_values('Symbol')

In [22]:
# add the necessary columns to the df for the data we will be extracting
columns_list = ['Symbol', 'Description', 'Quantity', 'Last Price', 'Last Price Change', 'Current Value', "Today's Gain/Loss Dollar", "Today's Gain/Loss Percent", "Total Gain/Loss Dollar", "Total Gain/Loss Percent", "Percent of Account", "Cost Basis Per Share", "Cost Basis Total", 'Div. Amt', 'Div. Yield', 'Equity Summary Score', 'Recognia Short Term', 'Recognia Intermediate Term', 'Recognia Long Term', 'Valuation', 'Quality', 'Growth Stability', 'Financial Health', '1-Yr Price Target', 'P/E TTM', 'P/E 5-Yr Avg', 'P/CF Recent Quarter', 'P/CF TTM', 'P/Sales Recent Quarter', 'P/Sales TTM', 'P/B', 'Debt/Equity TTM', 'Debt/Equity Recent Quarter', '52-Week Performance', "Account Name/Number"]
stock_df = stock_df.reindex(columns = columns_list)

In [ ]:
stock_df